In [1]:
data_dir = '/home/ubuntu/data/iceberg'

import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers.merge import Add, Concatenate
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
import cv2
import keras
import os
from keras.callbacks import TensorBoard

from keras import __version__
print(__version__)


Using TensorFlow backend.


2.1.1


In [2]:
np.random.seed(1234)

def get_scaled_imgs(df):
    imgs = []

    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

#         imgs.append(np.dstack((band_1, band_2, band_3)))
        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [3]:
# Additional image features from David
train_features = pd.read_csv("/home/ubuntu/data/iceberg/features/train.csv", index_col=0)
train_features.drop(labels=["is_iceberg", "inc_angle"], inplace=True, axis=1)
#test_features = pd.read_csv("iceberg_features/test.csv", index_col=0)

num_features = train_features.shape[1]


In [4]:
num_features

179

In [5]:
df_train = pd.read_json(os.path.join(data_dir, 'train.json'))

df_join = pd.merge(df_train, train_features, on='id', how="outer")

Xtrain = get_scaled_imgs(df_join)
Ytrain = np.array(df_join['is_iceberg'])

df_join.inc_angle = df_join.inc_angle.replace('na',0)
idx_tr = np.where(df_join.inc_angle>0)

Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]
Xfeatures = df_join.drop(labels=["id", "is_iceberg", "band_1", "band_2"], axis=1).iloc[idx_tr[0]]

inc_angle = Xfeatures.iloc[:,0:1]

In [6]:
inc_angle.shape

(1471, 1)

In [7]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
Xfeatures.iloc[:,1:179] = min_max_scaler.fit_transform(Xfeatures.iloc[:,1:179])
Xfeatures = pd.DataFrame(Xfeatures)


In [8]:
Xfeatures

,inc_angle,band1.0.m.cx,band1.0.m.cy,band1.0.m.majoraxis,band1.0.m.eccentricity,band1.0.m.theta,band1.0.s.area,band1.0.s.perimeter,band1.0.s.radius.mean,band1.0.s.radius.sd,...,band2.Ba.h.var.s2,band2.Ba.h.idm.s2,band2.Ba.h.sav.s2,band2.Ba.h.sva.s2,band2.Ba.h.sen.s2,band2.Ba.h.ent.s2,band2.Ba.h.dva.s2,band2.Ba.h.den.s2,band2.Ba.h.f12.s2,band2.Ba.h.f13.s2
0,43.9239,0.494756,0.495616,0.103221,0.941760,0.042102,0.010654,0.018265,0.085889,0.112781,...,0.602761,0.737116,0.821546,0.745714,0.379207,0.377427,0.000000,0.126811,0.014846,0.116523
1,38.1562,0.485197,0.532935,0.103426,0.951659,0.501300,0.009470,0.017352,0.079175,0.118316,...,0.004145,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,45.2859,0.511777,0.497717,0.044357,0.789014,0.089221,0.005031,0.008219,0.043723,0.046857,...,0.852684,0.538941,0.329327,0.196049,0.451034,0.489892,0.000000,0.187851,0.005754,0.082524
3,43.8306,0.562976,0.470446,0.034102,0.707574,0.105458,0.003847,0.006393,0.034812,0.041439,...,0.004145,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,35.6256,0.500823,0.450152,0.046437,0.687752,0.085616,0.006807,0.010046,0.054164,0.036080,...,0.004145,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,36.9034,0.468283,0.437232,0.138158,0.793867,0.293086,0.032554,0.029224,0.167948,0.100572,...,0.856995,0.596586,0.679501,0.570047,0.626598,0.615360,0.068169,0.322802,0.029238,0.208841
6,34.4751,0.500316,0.469714,0.197512,0.960440,0.180585,0.026931,0.032877,0.178466,0.221128,...,0.673442,0.545668,0.270833,0.166950,0.848921,0.817931,0.254438,0.706719,0.082374,0.403971
7,41.1769,0.483258,0.475220,0.170729,0.876345,0.911024,0.035809,0.037443,0.189673,0.158450,...,0.004145,1.000000,0.031250,0.000977,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,35.7829,0.608955,0.533824,0.130403,0.941166,0.970553,0.015981,0.023744,0.114970,0.138804,...,0.301488,0.727982,0.900095,0.829867,0.420391,0.413174,0.141144,0.334383,0.040803,0.202811
9,43.3007,0.507456,0.499572,0.023223,0.632850,0.864680,0.002664,0.004566,0.025690,0.022076,...,0.004145,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [189]:
# Create a train and validation split, 75% of data used in training
from sklearn.model_selection import train_test_split

X_train, X_valid, X_features_train, X_features_valid, Y_train, Y_valid = train_test_split(Xtrain,
                                    Xfeatures, Ytrain, random_state=666, train_size=0.85, test_size=0.15)

X_train.shape, X_valid.shape, X_features_train.shape, X_features_valid.shape, Y_train.shape, Y_valid.shape

((1250, 75, 75, 3), (221, 75, 75, 3), (1250, 179), (221, 179), (1250,), (221,))

In [10]:
X_train = get_more_images(X_train)
X_features_train = np.concatenate((X_features_train,X_features_train,X_features_train))
Y_train = np.concatenate((Y_train,Y_train,Y_train))

X_train.shape, X_features_train.shape, Y_train.shape

((3750, 75, 75, 3), (3750, 179), (3750,))

In [11]:
batch_size = 256
        
tbCallBack = TensorBoard(log_dir='/home/ubuntu/data/tensorboardlogs/', histogram_freq=0, write_graph=True, write_images=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, epsilon=1e-4, mode='min')

In [12]:
num_features

179

In [66]:
p = 0.2
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
features_input = Input(shape=(num_features,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
# x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)



# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([features_input, x])
m = Dense(512, activation='relu')(m)
x = Dropout(p)(x)
m = Dense(512, activation='relu')(m)
x = Dropout(p)(x)
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='sigmoid')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0001, decay=0.0)
model = Model(inputs=[classifier_input, features_input], outputs=out)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:

# model.fit([X_train, X_features_train], Y_train,
#                                           batch_size=128, 
#                                           epochs=50,
#                                           validation_data=([X_valid, X_features_valid], Y_valid),
#                                           shuffle=True, 
#                                           callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

In [69]:
model.fit([X_train, X_features_train], to_categorical(Y_train),
                                          batch_size=256, 
                                          epochs=50,
                                          validation_data=([X_valid, X_features_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 3750 samples, validate on 221 samples
Epoch 1/50
3750/3750 [==============================] - 6s 2ms/step - loss: 0.3856 - acc: 0.8216 - val_loss: 0.3150 - val_acc: 0.8846
Epoch 2/50
3750/3750 [==============================] - 6s 2ms/step - loss: 0.3449 - acc: 0.8396 - val_loss: 0.2882 - val_acc: 0.8824
Epoch 3/50
3750/3750 [==============================] - 6s 2ms/step - loss: 0.3298 - acc: 0.8504 - val_loss: 0.2606 - val_acc: 0.9050
Epoch 4/50
3750/3750 [==============================] - 6s 2ms/step - loss: 0.3105 - acc: 0.8621 - val_loss: 0.2626 - val_acc: 0.8801
Epoch 5/50
3750/3750 [==============================] - 6s 2ms/step - loss: 0.2891 - acc: 0.8717 - val_loss: 0.2456 - val_acc: 0.8914
Epoch 6/50
3750/3750 [==============================] - 6s 2ms/step - loss: 0.2769 - acc: 0.8761 - val_loss: 0.2249 - val_acc: 0.8959
Epoch 7/50
3750/3750 [==============================] - 6s 2ms/step - loss: 0.2724 - acc: 0.8800 - val_loss: 0.2172 - val_acc: 0.9027
Epoch 8/50
3750

In [82]:
from keras import backend as K
print(K.get_value(model.optimizer.lr))
K.set_value(model.optimizer.lr, .0001)
print(K.get_value(model.optimizer.lr))

0.001
0.0001


In [83]:
print(K.get_value(model.optimizer.lr))

0.0001


In [84]:
model.fit([X_train, X_features_train], to_categorical(Y_train),
                                          batch_size=256, 
                                          epochs=5,
                                          validation_data=([X_valid, X_features_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 3750 samples, validate on 221 samples
Epoch 1/5
3750/3750 [==============================] - 6s 2ms/step - loss: 0.2093 - acc: 0.9157 - val_loss: 0.2102 - val_acc: 0.9050
Epoch 2/5
3750/3750 [==============================] - 6s 2ms/step - loss: 0.1867 - acc: 0.9275 - val_loss: 0.2037 - val_acc: 0.9118
Epoch 3/5
3750/3750 [==============================] - 6s 2ms/step - loss: 0.1840 - acc: 0.9296 - val_loss: 0.2060 - val_acc: 0.9095
Epoch 4/5
3750/3750 [==============================] - 6s 2ms/step - loss: 0.1793 - acc: 0.9321 - val_loss: 0.2083 - val_acc: 0.9095
Epoch 5/5
3750/3750 [==============================] - 6s 2ms/step - loss: 0.1822 - acc: 0.9309 - val_loss: 0.2065 - val_acc: 0.9140


In [55]:
model.load_weights('/home/ubuntu/data/iceberg/results/weights/model_with_haralicks_094_acc.h5')

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 75, 75, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 37, 37, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 37, 37, 128)  73856       max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

In [ ]:
model.optimizertimizerodel.fit([X_train, X_features_train], Y_train,
                                          batch_size=128, 
                                          epochs=3,
                                          validation_data=([X_valid, X_features_valid], Y_valid),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

In [169]:
preds_dir = '/home/ubuntu/data/iceberg/results/final_preds/mix'
pred_test = pd.read_csv(preds_dir + '/cnn_train_0.96_best.csv').as_matrix()


In [28]:
model.save_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-Copy1_94_acc.h5')

In [173]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model.evaluate([Xtrain,Xfeatures], to_categorical(Ytrain), verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

# Additional image features from David
test_features = pd.read_csv("/home/ubuntu/data/iceberg/features/test.csv", index_col=0)
test_features.drop(labels=["inc_angle"], inplace=True, axis=1)
# #test_features = pd.read_csv("iceberg_features/test.csv", index_col=0)

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))

df_join_test = pd.merge(df_test, test_features, on='id', how="outer")

df_join_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_join_test))
Xtest_inc = df_join_test.inc_angle
Xtest_features = df_join_test.drop(labels=["id", "band_1", "band_2"], axis=1)


1471/1471 [==============================] - 1s 629us/step
Train score: 0.168463380644
Train accuracy: 0.938137320942


In [174]:
# pred_test = model.predict([Xtest,Xtest_features])

In [175]:
Xtest.shape, Xtest_inc.shape, Xtest_features.shape, pred_test.shape

((8424, 75, 75, 3), (8424,), (8424, 179), (8424, 2))

In [176]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
Xtest_features.iloc[:,1:179] = min_max_scaler.fit_transform(Xtest_features.iloc[:,1:179])
Xtest_features = pd.DataFrame(Xtest_features)

In [178]:
# Xtest_features

In [187]:
# Create a train and validation split, 75% of data used in training
from sklearn.model_selection import train_test_split

Xtest, X_valid2, Xtest_features, X_features_valid2, pred_test, Y_valid2 = train_test_split(Xtest,
                                    Xtest_features, pred_test, random_state=666, train_size=0.99, test_size=0.01)

Xtest.shape, X_valid2.shape, Xtest_features.shape, X_features_valid2.shape, pred_test.shape, Y_valid2.shape

((8172, 75, 75, 3),
 (83, 75, 75, 3),
 (8172, 1, 179),
 (83, 1, 179),
 (8172, 2),
 (83, 2))

In [181]:
# pseudo labelling using the above model

In [196]:
# this is the correct way round with [:,1]
idx_pred_1 = (np.where(pred_test[:,1]>0.9))
idx_pred_0 = (np.where(pred_test[:,1]<0.1))

In [195]:
Xtest_features.shape, X_features_train.shape

((8172, 1, 179), (1250, 1, 179))

In [137]:
type(Xtest_features)

pandas.core.frame.DataFrame

In [191]:
# Xtest_features= np.expand_dims(Xtest_features, axis=1)
X_features_train = np.expand_dims(X_features_train, axis=1)

In [100]:
Y_train = np.squeeze(Y_train)

In [97]:
Y_train = np.expand_dims(Y_train, axis=1)

In [194]:
X_train.shape, Y_train.shape, X_features_train.shape

((1250, 75, 75, 3), (1250,), (1250, 1, 179))

In [197]:
Xtrain_pl = np.concatenate((X_train, Xtest[idx_pred_1[0],...], Xtest[idx_pred_0[0],...]))
Ytrain_pl = np.concatenate((Y_train, np.ones(idx_pred_1[0].shape[0]), np.zeros(idx_pred_0[0].shape[0])))
Xfeatures_pl = np.concatenate((X_features_train, Xtest_features[idx_pred_1[0],...], Xtest_features[idx_pred_0[0],...]))

Xtrain_pl.shape, Ytrain_pl.shape, Xfeatures_pl.shape

((6389, 75, 75, 3), (6389,), (6389, 1, 179))

In [149]:
Xfeatures_pl

array([[  3.36383000e+01,   5.00588553e-01,   4.67765101e-01, ...,
          2.63998184e-01,   7.10061929e-02,   3.24395960e-01],
       [  3.95560000e+01,   5.06960400e-01,   4.83753482e-01, ...,
          2.47054153e-01,   3.24999004e-02,   1.93697333e-01],
       [  4.11227000e+01,   5.04139506e-01,   5.08525256e-01, ...,
          2.05253887e-01,   3.41996029e-02,   2.08852112e-01],
       ..., 
       [  4.32964000e+01,   5.20551724e-01,   5.53583540e-01, ...,
          1.83492614e-01,   1.47718640e-02,   1.39910668e-01],
       [  4.28744222e+01,   2.59100829e-01,   4.44639244e-01, ...,
          1.46653698e-01,   1.07354478e-01,   3.71863288e-01],
       [  3.74889000e+01,   5.46588408e-01,   5.33905141e-01, ...,
          1.71438966e-01,   6.78073835e-03,   8.38025895e-02]])

In [198]:
Xfeatures_pl = np.squeeze(Xfeatures_pl)

In [199]:
p = 0.2
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
features_input = Input(shape=(num_features,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
# x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)



# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([features_input, x])
m = Dense(512, activation='relu')(m)
x = Dropout(p)(x)
m = Dense(512, activation='relu')(m)
x = Dropout(p)(x)
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='sigmoid')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0001, decay=0.0)
model_p = Model(inputs=[classifier_input, features_input], outputs=out)
model_p.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [201]:
model_p.fit([Xtrain_pl, Xfeatures_pl], to_categorical(Ytrain_pl),
                                          batch_size=256, 
                                          epochs=50,
                                          validation_data=([X_valid, X_features_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 6389 samples, validate on 221 samples
Epoch 1/50
6389/6389 [==============================] - 10s 2ms/step - loss: 0.4222 - acc: 0.8062 - val_loss: 0.4386 - val_acc: 0.8054
Epoch 2/50
6389/6389 [==============================] - 10s 2ms/step - loss: 0.3940 - acc: 0.8255 - val_loss: 0.4124 - val_acc: 0.8213
Epoch 3/50
6389/6389 [==============================] - 10s 2ms/step - loss: 0.3584 - acc: 0.8425 - val_loss: 0.3678 - val_acc: 0.8462
Epoch 4/50
6389/6389 [==============================] - 10s 2ms/step - loss: 0.3216 - acc: 0.8627 - val_loss: 0.3142 - val_acc: 0.8620
Epoch 5/50
6389/6389 [==============================] - 10s 2ms/step - loss: 0.2812 - acc: 0.8832 - val_loss: 0.2830 - val_acc: 0.8643
Epoch 6/50
6389/6389 [==============================] - 10s 2ms/step - loss: 0.2606 - acc: 0.8924 - val_loss: 0.2612 - val_acc: 0.8801
Epoch 7/50
6389/6389 [==============================] - 10s 2ms/step - loss: 0.2416 - acc: 0.9021 - val_loss: 0.2513 - val_acc: 0.8801
Epoch 8/

In [207]:
from keras import backend as K
print(K.get_value(model_p.optimizer.lr))
K.set_value(model_p.optimizer.lr, .0001)
print(K.get_value(model_p.optimizer.lr))

0.001
0.0001


In [211]:
model_p.fit([Xtrain_pl, Xfeatures_pl], to_categorical(Ytrain_pl),
                                          batch_size=256, 
                                          epochs=2,
                                          validation_data=([X_valid, X_features_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 6389 samples, validate on 221 samples
Epoch 1/2
6389/6389 [==============================] - 10s 2ms/step - loss: 0.0365 - acc: 0.9890 - val_loss: 0.1485 - val_acc: 0.9502
Epoch 2/2
6389/6389 [==============================] - 10s 2ms/step - loss: 0.0363 - acc: 0.9887 - val_loss: 0.1491 - val_acc: 0.9457


In [ ]:
model.save_weights('/home/ubuntu/data/iceberg/results/weights/haralicks_model_acc95.h5')

In [213]:
score = model_p.evaluate([Xtrain,Xfeatures], to_categorical(Ytrain), verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

# Additional image features from David
test_features = pd.read_csv("/home/ubuntu/data/iceberg/features/test.csv", index_col=0)
test_features.drop(labels=["inc_angle"], inplace=True, axis=1)
# #test_features = pd.read_csv("iceberg_features/test.csv", index_col=0)

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))

df_join_test = pd.merge(df_test, test_features, on='id', how="outer")

df_join_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_join_test))
Xtest_inc = df_join_test.inc_angle
Xtest_features = df_join_test.drop(labels=["id", "band_1", "band_2"], axis=1)

pred_test = model_p.predict([Xtest,Xtest_features])

1471/1471 [==============================] - 1s 670us/step
Train score: 0.11513602976
Train accuracy: 0.961930658808


In [214]:
Xtest.shape, pred_test.shape

((8424, 75, 75, 3), (8424, 2))

In [215]:
submit_nums = np.clip(pred_test[:,1], 0.1, 0.9)

In [218]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': submit_nums.reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('cnn_with_haralicks_clipped' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id  is_iceberg
0  5941774d         0.1
1  4023181e         0.9
2  b20200e4         0.1
3  e7f018bb         0.1
4  4371c8c3         0.1
5  a8d9b1fd         0.9
6  29e7727e         0.1
7  92a51ffb         0.1
8  c769ac97         0.1
9  aee0547d         0.1


In [217]:
import os
os.getcwd()

'/home/ubuntu/git/learningWithKaggle/ice'

In [43]:
p = 0.2
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# x = BatchNormalization(axis=-1)(x)

# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0015, decay=0.0)
model2 = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
model2.fit([Xtrain_pl, Xinc_pl], to_categorical(Ytrain_pl),
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 9763 samples, validate on 148 samples
Epoch 1/50
9763/9763 [==============================] - 16s 2ms/step - loss: 0.6166 - acc: 0.6713 - val_loss: 0.6229 - val_acc: 0.7128
Epoch 2/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.3354 - acc: 0.8669 - val_loss: 0.5192 - val_acc: 0.7770
Epoch 3/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.2501 - acc: 0.9042 - val_loss: 0.4833 - val_acc: 0.7939 loss: 0.2537 - acc
Epoch 4/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.2342 - acc: 0.9029 - val_loss: 0.2614 - val_acc: 0.8716
Epoch 5/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.1608 - acc: 0.9301 - val_loss: 0.2916 - val_acc: 0.8649
Epoch 6/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.1422 - acc: 0.9369 - val_loss: 0.2314 - val_acc: 0.8986
Epoch 7/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.1399 - acc: 0.9404 - val_loss: 0.2208 - val_a

In [ ]:
# as well as adding the pseudo labelling, augment the pseudo labelled data?

In [45]:
# this way will reaugment the orginal images, perhaps doesn't matter? 

Xtrain_pl = get_more_images(Xtrain_pl)
Xinc_pl = np.concatenate((Xinc_pl,Xinc_pl,Xinc_pl))
Ytrain_pl = np.concatenate((Ytrain_pl,Ytrain_pl,Ytrain_pl))

Xtrain_pl.shape, Ytrain_pl.shape, Xinc_pl.shape

((29289, 75, 75, 3), (29289,), (29289,))

In [46]:
model2.fit([Xtrain_pl, Xinc_pl], to_categorical(Ytrain_pl),
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 29289 samples, validate on 148 samples
Epoch 1/50
29289/29289 [==============================] - 46s 2ms/step - loss: 0.0408 - acc: 0.9864 - val_loss: 0.1343 - val_acc: 0.9561
Epoch 2/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0367 - acc: 0.9876 - val_loss: 0.1561 - val_acc: 0.9459
Epoch 3/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0316 - acc: 0.9882 - val_loss: 0.1464 - val_acc: 0.9662
Epoch 4/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0343 - acc: 0.9881 - val_loss: 0.1358 - val_acc: 0.9595 acc: 0.98 - ETA: 5s - loss: 0.0344 - acc: 0.988 - ETA: 4s - loss: 0.0344 - acc: 0.987 - ETA: 4s - lo
Epoch 5/50
29289/29289 [==============================] - 46s 2ms/step - loss: 0.0291 - acc: 0.9892 - val_loss: 0.1477 - val_acc: 0.9595
Epoch 6/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0299 - acc: 0.9893 - val_loss: 0.1663 - val_acc: 0.9527
Epoch 7/50
29289/29289 [=====

In [16]:
model.load_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-Copy1_model2_95_acc.h5')

ValueError: Dimension 0 in both shapes must be equal, but are 435 and 257 for 'Assign_10' (op: 'Assign') with input shapes: [435,512], [257,512].

In [36]:
Xtest.shape, Xtest_inc.shape, pred_test.shape

NameError: name 'Xtest' is not defined

In [51]:
Xtest = get_more_images(Xtest)
Xtest_inc = np.concatenate((Xtest_inc,Xtest_inc,Xtest_inc))
pred_test = np.concatenate((pred_test, pred_test, pred_test))

Xtest.shape, Xtest_inc.shape, pred_test.shape

((25272, 75, 75, 3), (25272,), (25272, 2))

In [52]:
# this is the correct way round with [:,1]
idx_pred_1 = (np.where(pred_test[:,1]>0.95))
idx_pred_0 = (np.where(pred_test[:,1]<0.05))

In [57]:
# Xtest_inc = np.expand_dims(Xtest_inc, axis=1)
X_inc_train = np.expand_dims(X_inc_train, axis=1)

In [58]:
# X_inc_train = np.squeeze(X_inc_train)
X_inc_train.shape

(3969, 1)

In [59]:
Xtrain_pl = np.concatenate((X_train, Xtest[idx_pred_1[0],...], Xtest[idx_pred_0[0],...]))
Ytrain_pl = np.concatenate((Y_train, np.ones(idx_pred_1[0].shape[0]), np.zeros(idx_pred_0[0].shape[0])))
Xinc_pl = np.concatenate((X_inc_train, Xtest_inc[idx_pred_1[0],...], Xtest_inc[idx_pred_0[0],...]))

Xtrain_pl.shape, Ytrain_pl.shape, Xinc_pl.shape

((21351, 75, 75, 3), (21351,), (21351, 1))

In [34]:
p = 0.4
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# x = BatchNormalization(axis=-1)(x)

# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p/2)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0015, decay=0.0)
model3 = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model3.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
model3.fit([Xtrain_pl, Xinc_pl], to_categorical(Ytrain_pl),
                                          batch_size=256, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

NameError: name 'Xtrain_pl' is not defined

In [62]:
model2.save_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-Copy1_model2_95_acc.h5')

In [106]:
# submit_nums = np.clip(pred_test[:,1], 0.1, 0.9)
submit_nums[1]

0.89999998

In [109]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('cnn_train_' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id  is_iceberg
0  5941774d    0.001687
1  4023181e    0.952506
2  b20200e4    0.778264
3  e7f018bb    0.997760
4  4371c8c3    0.273599
5  a8d9b1fd    0.829154
6  29e7727e    0.055702
7  92a51ffb    0.998996
8  c769ac97    0.000014
9  aee0547d    0.000027


In [92]:
df_test.shape

(8424, 4)

In [114]:
model.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)

4413/4413 [==============================] - 3s 600us/step


[0.13649047835023664, 0.95014729196998826]